<center>
<h1> D ONE MLOps AWS BlogPost </h1>
    <h2>Work In Progress</h2>
<hr>
<h1>XGBoost Trainig Pipeline</h1>
<hr>
 </center>

<h2>Problem Introduction</h2>

* See EDA notebook
* Add a few high level lines here

# Setting up shop

A known challenge with SageMaker Studio default libraries is that they are often outdated. In this case, we want to make sure that pandas and s3fs is up-to-date. s3fs is a file system interface for S3 (https://pypi.org/project/s3fs/).

In [83]:
! pip install pandas s3fs --upgrade

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached s3fs-2022.5.0-py3-none-any.whl (27 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 1.7 MB/s eta 0:00:00a 0:00:01
  Using cached aiobotocore-2.3.2.tar.gz (104 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [20 lines of output]
      Traceback (most recent call last):
        File "<string>", line 36, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-pcd02ia7/aiobotocore_0c3fb1baad4f44299

# Import packages
We need a few packages, let's import them.

In [71]:
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

import boto3  # Amazon Web Services (AWS) Software Development Kit (SDK) for Python
import os
import pandas as pd
import sagemaker
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

pd.options.mode.chained_assignment = None  # default='warn'

# Set up global variables

In [72]:
# This is the role that SageMaker assumes to leverage AWS resources
ROLE = sagemaker.get_execution_role()
ROLE

'arn:aws:iam::451811961115:role/AmazonSageMaker-ExecutionRole-20200526T075798'

In [73]:
# Paths for the containers that we will create
# S3 Bucket where the data is stored
BUCKET_NAME = "aws-sagemaker-blogpost"
BUCKET = f's3://{BUCKET_NAME}'

# Raw data paths
RAW_DATA_FOLDER = 'data'
# Note if you change the line below, you also need to change it in the cell that creates the prepare_data.py
RAW_DATA_FILE = 'wind_turbines.csv'
RAW_DATA_PATH = os.path.join(BUCKET, RAW_DATA_FOLDER, RAW_DATA_FILE)

# Path where the processed objects will be stored
now = datetime.now() # get current time to ensure uniqueness of the output folders
PROCESSED_DATA_FOLDER = 'processed_' + now.strftime("%Y-%m-%d_%H%M_%S%f")
PROCESSED_DATA_PATH = os.path.join(BUCKET, PROCESSED_DATA_FOLDER)

# Paths for model train, validation, test split
TRAIN_DATA_PATH = os.path.join(PROCESSED_DATA_PATH, 'train.csv')
VALIDATION_DATA_PATH = os.path.join(PROCESSED_DATA_PATH, 'validation.csv')
TEST_DATA_PATH = os.path.join(PROCESSED_DATA_PATH, 'test.csv')

# Path to model output data
MODEL_OUTPUT = os.path.join(BUCKET, 'output')

In [74]:
# Job name for preprocessing
PREPROCESSING_JOB_NAME = 'windTurbinesPreprocessing'

In [75]:
df = pd.read_csv(RAW_DATA_PATH)
df.loc[:, 'categories_sk'].value_counts(dropna=False)

NaN     49416
5.0      2415
20.0      235
3.0       175
19.0      123
24.0       10
22.0        9
Name: categories_sk, dtype: int64

In [76]:
df.loc[:, 'subtraction'].value_counts(dropna=False)

NaN    49416
1.0     2424
0.0      543
Name: subtraction, dtype: int64

# Preprocess data (TODO)

The data comes in chunks, but for our purposes, we would like to have one large file with data. We could also extend the code to read only new data and add it to the file, but for simplicity, let's load the whole dataset for processing.

Note: Having the data in chunks would allow for online processing of new data when it comes in, e.g., when a turbine reports the next couple of months data.

For details on the dataset, see the exploratory data analysis work in Part 1.

* TODO: Expand
* TODO: Link to EDA.

Amazon SageMaker XGBoost can train on data in either a CSV or LibSVM format. Here, we stick to CSV. The requirements on the data formatting are:

1. The target variable must be in the first column.
2. There cannot be a header row.

We first create a preprocessing script that will be used by a built-in SKLearn container to run the preprocessing job of our pipeline. Running the cell below will produce a Python file whose content is the same as the cell's content and named `prepare_data.py`. It will be saved in the notebooks instance path (see left pane of the SageMaker Studio JupyterLab).

We then define the container that will read in the data from the S3 bucket and use this script to preprocess it and then write the processed files back to the S3 bucket.

In [77]:
%%writefile prepare_data.py

import argparse
import logging
import os
import pandas as pd
from datetime import datetime, timedelta
from typing import List, Tuple, Union

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())

pd.options.mode.chained_assignment = None  # default='warn'


def assert_col_of_df(df: pd.DataFrame, col: Union[List[str], str]) -> None:
    """Helper function to assert that a column `col` is a column of `df`.
    
    Args:
        df: Dataframe.
        col: String value to test.
    
    Returns:
        None.
        
    Raises:
        ValueError if `col` is not a column of `df`.
    """
    if isinstance(col, str):
        col = [col]

    for c in col:
        try:
            assert c in df.columns      
        except AssertionError:
            raise ValueError(f"Invalid input value. Column {c} is not a column of df.")

            
def get_train_test_split(
        df: pd.DataFrame,
        n_days_test: int
        ) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Splits the input data frame into a training and test set.

    Args:
        df: Raw input data.
        n_days_test: Number of days to consider for the test split. The n_days_test last 
            days of the input data will be selected for the test split.

    Returns:
        Tuple[pd.DataFrame]: Raw train and test data splits.
    """
    _date_col = 'date'
    _measured_at_col = 'measured_at'
    
    assert_col_of_df(df=df, col=_measured_at_col)
    
    # Take only the date part of the string, i.e., the first 10 characters
    df[_date_col] = df[_measured_at_col].apply(lambda x: x[:10])
    # Convert to date object
    df[_date_col] = pd.to_datetime(df[_date_col], format='%Y-%m-%d')
    
    # Get the test dates
    min_date = df[_date_col].min()
    max_date = df[_date_col].max()
    
    test_dates = [
        datetime.strftime(max_date - timedelta(days=i), '%Y-%m-%d') for i in range(n_days_test)
    ]
    
    df_train = df[~df[_date_col].isin(test_dates)].drop(_date_col, axis=1)
    df_test = df[df[_date_col].isin(test_dates)].drop(_date_col, axis=1)
    
    logger.info(f"Train set ranges from {min_date} until {min(test_dates)} (not included).")
    logger.info(f"Test set ranges from {min(test_dates)} until {max(test_dates)}.")
    
    return df_train, df_test


def fill_nulls(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """Fills nulls in column `col` of dataframe `df`.
    
    Args:
        df: Raw input dataframe.
        col: Column of `df` with nulls filled with 0.
        
    Returns:
        Dataframe with nulls filled.
    """
    assert_col_of_df(df=df, col=col)
        
    df.loc[:, col] = (
        df
        .loc[:, col]
        .fillna(0)
    )
    
    logger.info(f"Filled nulls in column {col} with 0.")

    return df


def filter_power(df: pd.DataFrame, col_power: str, min_power: float) -> pd.DataFrame:
    """Filters the `df` on the power column `col_power`.
    
    
    Args:
        df: Raw input dataframe.
        col_power: Column of `df` with the power production.
        min_power: Minimum values of power production considered. Rows with smaller
            values are filtered out.
        
    Returns:
        Dataframe filtered on `min_power`.
    """
    assert_col_of_df(df=df, col=col_power)
    
    filter_power = df[col_power] > min_power
    
    rowcount_before = df.shape[0]
    df = (
        df
        .loc[filter_power]
    )
    rowcount_after = df.shape[0]
    logger.info(f"Removed {rowcount_before-rowcount_after} rows which had power below {min_power}.")
    
    return df


def split_features_target(
    df: pd.DataFrame,
    features: List[str],
    target: str
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Function to split the dataframe into input features and labels.
    
    Args:
        df: Raw input data.
        features: List of the features to be included in the transformation.
        target: Target column.
    
    Returns:
        Tuple[pd.DataFrame]: Transformed dataframes (input features and labels).
    """
    assert_col_of_df(df=df, col=(features + [target]))

    y = df.loc[:, target]
    x = df.loc[:, features]
    logger.info(f"Split dataframe into features with shape {x.shape} and target with shape {y.shape}.")
    return x, y


def wrap_transform_data(
    df: pd.DataFrame,
    col_power: str,
    min_power: float,
    col_errors: str,
    errors_to_classify: List[int],
    features: List[str],
    target: str
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Wrapper for transforming the data for the model
    
    Processing is applied in the following steps:
        1. Filtering out low power values
        2. Fill nulls in all of the feature columns
        3. Split dataset into features and target (x, y)
        
    Args:
        df: Input dataframe.
        col_power: Column of `df` with the power production.
        min_power: Minimum values of power production considered. Rows with smaller
            values are filtered out.
        features: List of the features to be included in the transformation.
        target: Target column.
    
    Returns:
        
    """
    # 1. Filter out low power
    df = filter_power(df=df, col_power=col_power, min_power=min_power)

    # 2. Fill nulls in all of the feature columns
    for feat in features:
        df = fill_nulls(df=df, col=feat)

    # 3. Split dataset into features and target
    x, y = split_features_target(df=df, features=features, target=target) 
    
    return x, y


# ----- CONSTANTS ----- #
# Columns of df
# Error column <> target
COL_ERRORS = 'subtraction'
# Power produced column (used for filtering out small values)
COL_POWER = 'power'
# Features to consider for the model
FEATURES = ['wind_speed', 'power', 'nacelle_direction', 'wind_direction',
            'rotor_speed', 'generator_speed', 'temp_environment',
            'temp_hydraulic_oil', 'temp_gear_bearing', 'cosphi',
            'blade_angle_avg', 'hydraulic_pressure']
# Power values to filter out
MIN_POWER = 0.05
# Filname of the raw data file
RAW_DATA_FILE = 'wind_turbines.csv'


if __name__ == '__main__':
    
    logger.info(f'Preprocessing job started.')
    # Parse the SDK arguments that are passed when creating the SKlearn container
    parser = argparse.ArgumentParser()
    parser.add_argument("--n_test_days", type=int, default=10)
    parser.add_argument("--n_val_days", type=int, default=10)
    args, _ = parser.parse_known_args()

    logger.info(f"Received arguments {args}.")

    # Read in data locally in the container
    input_data_path = os.path.join("/opt/ml/processing/input", RAW_DATA_FILE)
    logger.info(f"Reading input data from {input_data_path}")
    # Read raw input data
    df = pd.read_csv(input_data_path)
    logger.info(f"Shape of data is: {df.shape}")

    # ---- Preprocess the data set ----
    logger.info("Split data into training+validation and test set.")
    df_train_valid, df_test = get_train_test_split(df=df, n_days_test=args.n_test_days) 

    logger.info("Split training+validation into training and validation set.")
    df_train, df_val = get_train_test_split(df=df_train_valid, n_days_test=args.n_val_days) 

    logger.info("Transforming training data.")
    x_train, y_train = wrap_transform_data(
        df=df_train,
        col_power=COL_POWER,
        min_power=MIN_POWER,
        col_errors=COL_ERRORS,
        features=FEATURES,
        target=COL_ERRORS
    )
    
    logger.info("Transforming validation data.")
    x_val, y_val = wrap_transform_data(
        df=df_val,
        col_power=COL_POWER,
        min_power=MIN_POWER,
        col_errors=COL_ERRORS,
        features=FEATURES,
        target=COL_ERRORS
    )

    # Create local output directories. These directories live on the container that is spun up.
    try:
        os.makedirs("/opt/ml/processing/train")
        os.makedirs("/opt/ml/processing/validation")
        os.makedirs("/opt/ml/processing/test")
        print("Successfully created directories")
    except Exception as e:
        # if the Processing call already creates these directories (or directory otherwise cannot be created)
        logger.debug(e)
        logger.debug("Could Not Make Directories.")
        pass

    # Save data locally on the container that is spun up.
    try:
        pd.DataFrame(x_train).to_csv("/opt/ml/processing/train/x_train.csv", header=True, index=False)
        pd.DataFrame(y_train).to_csv("/opt/ml/processing/train/y_train.csv", header=True, index=False)
        pd.DataFrame(x_val).to_csv("/opt/ml/processing/validation/x_val.csv", header=True, index=False)
        pd.DataFrame(y_val).to_csv("/opt/ml/processing/validation/y_val.csv", header=True, index=False)
        pd.DataFrame(df_test).to_csv("/opt/ml/processing/test/test.csv", header=True, index=False)
        logger.info("Files Successfully Written Locally")
    except Exception as e:
        logger.debug("Could Not Write the Files")
        logger.debug(e)
        pass

    logger.info("Finished running processing job")

Overwriting prepare_data.py


If one reads the script carefully one can see that the training, validation, and test datasets are stored in the file path `/opt/ml/processing/...`. This is not the path in the S3 bucket, but refers to the local path on the container that is spun up.

We specified that in the beginning when setting up path variables (e.g., `TEST_DATA_PATH`)! This will become useful later.
The first line of the cell's output will indicate the job's name. Go back to the AWS SageMaker Console and check the processing jobs. In the author's case - eu-central-1 AZ - the link is https://eu-central-1.console.aws.amazon.com/sagemaker/home?region=eu-central-1#/processing-jobs.

From there, one can leverage CloudWatch to check the logs for the job or make use of other monitoring tools - the full power of AWS can be leveraged.

In [78]:
%%time

base_job_name = PREPROCESSING_JOB_NAME
sklearn_processor = SKLearnProcessor(
    base_job_name=base_job_name,
    framework_version="0.20.0",
    role=ROLE,
    instance_type="ml.m5.xlarge",
    instance_count=1
)

sklearn_processor.run(
    code="prepare_data.py",
    inputs=[ProcessingInput(source=RAW_DATA_PATH, destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(
            destination=PROCESSED_DATA_PATH,
            source="/opt/ml/processing/train"),
        ProcessingOutput(
            destination=PROCESSED_DATA_PATH,
            source="/opt/ml/processing/validation"),
        ProcessingOutput(
            destination=PROCESSED_DATA_PATH,
            source="/opt/ml/processing/test"),
    ],
    arguments=["--n_test_days", "20",
              "--n_val_days", "30"],
)

preprocessing_job_description = sklearn_processor.jobs[-1].describe()


Job Name:  windTurbinesPreprocessing-2022-06-02-15-43-12-114
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://aws-sagemaker-blogpost/data/wind_turbines.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-eu-central-1-451811961115/windTurbinesPreprocessing-2022-06-02-15-43-12-114/input/code/prepare_data.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output-1', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://aws-sagemaker-blogpost/processed_2022-06-02_1543_09970865', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'output-2', 'AppManaged': False, 'S3Output': {'S3Uri

# Training

For this classification task, we use a bult-in XGBoost container provided by SageMaker. With `sagemaker.image_uri` we specify the training container image URI. In our case, the SageMaker XGBoost training container URI is specified using `sagemaker.image_uris.retrieve`.

You can find information on pre-built containers for example here
https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html or by exploring the SageMaker example notebooks.

In [82]:
region_name = boto3.Session().region_name
container = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region_name,
    version="1.0-1",
    py_version="py3"
)

print(f'Using SageMaker container: {container} in region {region_name}')

Using SageMaker container: 492215442770.dkr.ecr.eu-central-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3 in region eu-central-1


Define the SageMaker Estimator with resource configurations and hyperparameters to train a linear regressor on an c4.4xlarge instance. For different types of EC2 instances available from SageMaker, see https://aws.amazon.com/ec2/instance-explorer/?ec2-instances-cards.sort-by=item.additionalFields.category-order&ec2-instances-cards.sort-order=asc&awsf.ec2-instances-filter-category=*all&awsf.ec2-instances-filter-processors=*all&awsf.ec2-instances-filter-accelerators=*all&awsf.ec2-instances-filter-capabilities=*all

Specifying the `rules` with `create_xgboost_report()` creates an XGBoost report that provides insights into the training progress and results (see more information here: https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-training-xgboost-report.html).


In [84]:
TRAIN_DATA_PATH

's3://aws-sagemaker-blogpost/processed_2022-06-02_1543_09970865/train.csv'

In [ ]:
sess = sagemaker.Session()
xgboost_model = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.c4.4xlarge',
    output_path=MODEL_OUTPUT,
    sagemaker_session=sess,
    rules=[Rule.sagemaker(rule_configs.create_xgboost_report())]
)

# set the model's hyperparameters
xgboost_model.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0,
)

In [88]:
x_train = 's3://aws-sagemaker-blogpost/processed_2022-06-02_1543_09970865/x_train.csv'
y_train = 's3://aws-sagemaker-blogpost/processed_2022-06-02_1543_09970865/y_train.csv'
x_train = pd.read_csv(x_train)
y_train = pd.read_csv(y_train)
y_train.head()

,categories_sk
0,0
1,0
2,0
3,0
4,0


In [89]:
y_train.value_counts()

categories_sk
0                19443
5                  959
3                    4
9                    3
dtype: int64

In [90]:
x_train['subtraction']

KeyError: 'subtraction'